In [3]:
import os
import glob

import pandas as pd
import numpy as np

import statsmodels.tsa.stattools as smts
import statsmodels.api as sm    

from matplotlib import pyplot as plt

In [4]:
def price_spread_is_stationary(p1, p2, sig_level = 0.05):
    Y, X = pd.Series(p1), pd.Series(p2)
    adf_pvalue = smts.adfuller(Y - X)[1]

    return adf_pvalue < sig_level

def log_price_spread_is_stationary(p1, p2, sig_level = 0.05):
    Y, X = pd.Series(np.log(p1)), pd.Series(np.log(p2)) 
    adf_pvalue = smts.adfuller(Y - X)[1]

    return adf_pvalue < sig_level

In [61]:
df1 = pd.read_csv('../ib-data/iex-features/GOOGL.csv')
df2 = pd.read_csv('../ib-data/iex-features/GOOG.csv')

In [9]:
# price_spread_is_stationary(df1['close'][0:2000], df2['close'][0:2000])
# log_price_spread_is_stationary(df1['close'][0:2000], df2['close'][0:2000])

In [5]:
# log Y = intercept + log X + c
def compute_intercept_and_pvalue(p1, p2):
    Y, X = pd.Series(np.log(p1)), pd.Series(np.log(p2))
    
    # returns (intercept, pvalue)
    return (np.mean(Y - X), smts.adfuller(Y - X)[1])

def generate_pair_df(df1, df2, training_period = 84*20):
    dt1, dt2 = df1['date'][0], df2['date'][0]
    dt = max(dt1, dt2)
    
    p1_start, p2_start = df1[df1['date'] >= dt].index[0], df2[df2['date'] >= dt].index[0]
    
    df1_train, df1_test = df1[p1_start:p1_start + training_period], df1[p1_start + training_period:]
    df2_train, df2_test = df2[p2_start:p2_start + training_period], df2[p2_start + training_period:]
    
    intercept, pvalue = compute_intercept_and_pvalue(df1_train['close'], df2_train['close'])
    
    df_combined = pd.DataFrame()
    
    # date
    df_combined["date"] = df1_test["date"]
    
    # datetime features
    df_combined['year'] = df1_test['year']
    df_combined['monthOfYear'] = df1_test['monthOfYear']
    df_combined['dayOfMonth'] = df1_test['dayOfMonth']
    df_combined['hourOfDay'] = df1_test['hourOfDay']
    df_combined['minuteOfHour'] = df1_test['minuteOfHour']
    df_combined['dayOfWeek'] = df1_test['dayOfWeek']
    df_combined['dayOfYear'] = df1_test['dayOfYear']
    df_combined['weekOfYear'] = df1_test['weekOfYear']
    df_combined['isHoliday'] = df1_test['isHoliday']
    df_combined['prevDayIsHoliday'] = df1_test['prevDayIsHoliday']
    df_combined['nextDayIsHoliday'] = df1_test['nextDayIsHoliday']

    # spread and pvalue
    df_combined["spread"] = pd.Series(np.log(df1_test['close']) - np.log(df2_test['close']) - intercept)
    df_combined["pvalue"] = pvalue
    
    # price information of both stocks
    df_combined["open1"] = df1_test["open"]
    df_combined["high1"] = df1_test["high"]
    df_combined["low1"] = df1_test["low"]
    df_combined["close1"] = df1_test["close"]
                
    df_combined["open2"] = df2_test["open"]
    df_combined["high2"] = df2_test["high"]
    df_combined["low2"] = df2_test["low"]
    df_combined["close2"] = df2_test["close"]
    
    
    return df_combined  

In [63]:
compute_intercept_and_pvalue(df1['close'][0:100], df2['close'][0:100])

(0.010596148788664595, 0.098684168451024346)

In [64]:
df3 = generate_pair_df(df1, df2)
df3.to_csv(path_or_buf="../ib-data/pair-features/GOOGL-GOOG.csv", index=False)

## Generate pair_df for all NC2 pairs

In [6]:
nyse_csv_paths = []
nyse_csv_paths = sorted(glob.glob("../ib-data/nyse-daily-features/*.csv"))

N_STOCKS = len(nyse_csv_paths)
print(N_STOCKS)

170


In [9]:
data = {}
MIN_SIZE = 252*4
N_STOCKS_TAKEN = 0

for path in nyse_csv_paths:
    # get the file names
    filename = os.path.basename(path)
    filename_without_ext = os.path.splitext(filename)[0]
    
    # read the csv file as dataframe
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    
    # if price history is long enough, we take it
    if len(df) >= MIN_SIZE:
        data[filename_without_ext] = df 
        N_STOCKS_TAKEN += 1

In [10]:
print("N_STOCKS = " + str(N_STOCKS))
print("N_STOCKS_TAKEN = " + str(N_STOCKS_TAKEN))

N_STOCKS = 170
N_STOCKS_TAKEN = 116


In [11]:
# get max starting date
MAX_DATE = pd.Timestamp.min

for key in data.keys():
    # reset index 
    data[key] = data[key].reset_index(drop=True)
    
    # max
    MAX_DATE = max(MAX_DATE, data[key]['date'][0])
    
# take subset of all dataframes
for key in data.keys():
    data[key] = data[key][data[key]['date'] >= MAX_DATE]
    
    # reset index 
    data[key] = data[key].reset_index(drop=True)

In [17]:
import itertools

stocks = list(data.keys())
stock_pairs = list(itertools.combinations(stocks, 2))

In [23]:
for i in range(len(stock_pairs)):
    stk1, stk2 = stock_pairs[i]
    df1, df2 = data[stk1], data[stk2]
    
    # set training period to 60 days
    df3 = generate_pair_df(df1, df2, 60)
    
    PATH = stk1 + "-" + stk2 
    df3.to_csv(path_or_buf="../ib-data/pair-features-daily/" + PATH + ".csv", index=False)
    
    # print message
    print( str(i + 1) + "/" + str(len(stock_pairs)) + " completed.")

1/6670 completed.
2/6670 completed.
3/6670 completed.
4/6670 completed.
5/6670 completed.
6/6670 completed.
7/6670 completed.
8/6670 completed.
9/6670 completed.
10/6670 completed.
11/6670 completed.
12/6670 completed.
13/6670 completed.
14/6670 completed.
15/6670 completed.
16/6670 completed.
17/6670 completed.
18/6670 completed.
19/6670 completed.
20/6670 completed.
21/6670 completed.
22/6670 completed.
23/6670 completed.
24/6670 completed.
25/6670 completed.
26/6670 completed.
27/6670 completed.
28/6670 completed.
29/6670 completed.
30/6670 completed.
31/6670 completed.
32/6670 completed.
33/6670 completed.
34/6670 completed.
35/6670 completed.
36/6670 completed.
37/6670 completed.
38/6670 completed.
39/6670 completed.
40/6670 completed.
41/6670 completed.
42/6670 completed.
43/6670 completed.
44/6670 completed.
45/6670 completed.
46/6670 completed.
47/6670 completed.
48/6670 completed.
49/6670 completed.
50/6670 completed.
51/6670 completed.
52/6670 completed.
53/6670 completed.
54

418/6670 completed.
419/6670 completed.
420/6670 completed.
421/6670 completed.
422/6670 completed.
423/6670 completed.
424/6670 completed.
425/6670 completed.
426/6670 completed.
427/6670 completed.
428/6670 completed.
429/6670 completed.
430/6670 completed.
431/6670 completed.
432/6670 completed.
433/6670 completed.
434/6670 completed.
435/6670 completed.
436/6670 completed.
437/6670 completed.
438/6670 completed.
439/6670 completed.
440/6670 completed.
441/6670 completed.
442/6670 completed.
443/6670 completed.
444/6670 completed.
445/6670 completed.
446/6670 completed.
447/6670 completed.
448/6670 completed.
449/6670 completed.
450/6670 completed.
451/6670 completed.
452/6670 completed.
453/6670 completed.
454/6670 completed.
455/6670 completed.
456/6670 completed.
457/6670 completed.
458/6670 completed.
459/6670 completed.
460/6670 completed.
461/6670 completed.
462/6670 completed.
463/6670 completed.
464/6670 completed.
465/6670 completed.
466/6670 completed.
467/6670 completed.


829/6670 completed.
830/6670 completed.
831/6670 completed.
832/6670 completed.
833/6670 completed.
834/6670 completed.
835/6670 completed.
836/6670 completed.
837/6670 completed.
838/6670 completed.
839/6670 completed.
840/6670 completed.
841/6670 completed.
842/6670 completed.
843/6670 completed.
844/6670 completed.
845/6670 completed.
846/6670 completed.
847/6670 completed.
848/6670 completed.
849/6670 completed.
850/6670 completed.
851/6670 completed.
852/6670 completed.
853/6670 completed.
854/6670 completed.
855/6670 completed.
856/6670 completed.
857/6670 completed.
858/6670 completed.
859/6670 completed.
860/6670 completed.
861/6670 completed.
862/6670 completed.
863/6670 completed.
864/6670 completed.
865/6670 completed.
866/6670 completed.
867/6670 completed.
868/6670 completed.
869/6670 completed.
870/6670 completed.
871/6670 completed.
872/6670 completed.
873/6670 completed.
874/6670 completed.
875/6670 completed.
876/6670 completed.
877/6670 completed.
878/6670 completed.


1229/6670 completed.
1230/6670 completed.
1231/6670 completed.
1232/6670 completed.
1233/6670 completed.
1234/6670 completed.
1235/6670 completed.
1236/6670 completed.
1237/6670 completed.
1238/6670 completed.
1239/6670 completed.
1240/6670 completed.
1241/6670 completed.
1242/6670 completed.
1243/6670 completed.
1244/6670 completed.
1245/6670 completed.
1246/6670 completed.
1247/6670 completed.
1248/6670 completed.
1249/6670 completed.
1250/6670 completed.
1251/6670 completed.
1252/6670 completed.
1253/6670 completed.
1254/6670 completed.
1255/6670 completed.
1256/6670 completed.
1257/6670 completed.
1258/6670 completed.
1259/6670 completed.
1260/6670 completed.
1261/6670 completed.
1262/6670 completed.
1263/6670 completed.
1264/6670 completed.
1265/6670 completed.
1266/6670 completed.
1267/6670 completed.
1268/6670 completed.
1269/6670 completed.
1270/6670 completed.
1271/6670 completed.
1272/6670 completed.
1273/6670 completed.
1274/6670 completed.
1275/6670 completed.
1276/6670 com

1620/6670 completed.
1621/6670 completed.
1622/6670 completed.
1623/6670 completed.
1624/6670 completed.
1625/6670 completed.
1626/6670 completed.
1627/6670 completed.
1628/6670 completed.
1629/6670 completed.
1630/6670 completed.
1631/6670 completed.
1632/6670 completed.
1633/6670 completed.
1634/6670 completed.
1635/6670 completed.
1636/6670 completed.
1637/6670 completed.
1638/6670 completed.
1639/6670 completed.
1640/6670 completed.
1641/6670 completed.
1642/6670 completed.
1643/6670 completed.
1644/6670 completed.
1645/6670 completed.
1646/6670 completed.
1647/6670 completed.
1648/6670 completed.
1649/6670 completed.
1650/6670 completed.
1651/6670 completed.
1652/6670 completed.
1653/6670 completed.
1654/6670 completed.
1655/6670 completed.
1656/6670 completed.
1657/6670 completed.
1658/6670 completed.
1659/6670 completed.
1660/6670 completed.
1661/6670 completed.
1662/6670 completed.
1663/6670 completed.
1664/6670 completed.
1665/6670 completed.
1666/6670 completed.
1667/6670 com

...

4763/6670 completed.
4764/6670 completed.
4765/6670 completed.
4766/6670 completed.
4767/6670 completed.
4768/6670 completed.
4769/6670 completed.
4770/6670 completed.
4771/6670 completed.
4772/6670 completed.
4773/6670 completed.
4774/6670 completed.
4775/6670 completed.
4776/6670 completed.
4777/6670 completed.
4778/6670 completed.
4779/6670 completed.
4780/6670 completed.
4781/6670 completed.
4782/6670 completed.
4783/6670 completed.
4784/6670 completed.
4785/6670 completed.
4786/6670 completed.
4787/6670 completed.
4788/6670 completed.
4789/6670 completed.
4790/6670 completed.
4791/6670 completed.
4792/6670 completed.
4793/6670 completed.
4794/6670 completed.
4795/6670 completed.
4796/6670 completed.
4797/6670 completed.
4798/6670 completed.
4799/6670 completed.
4800/6670 completed.
4801/6670 completed.
4802/6670 completed.
4803/6670 completed.
4804/6670 completed.
4805/6670 completed.
4806/6670 completed.
4807/6670 completed.
4808/6670 completed.
4809/6670 completed.
4810/6670 com

5154/6670 completed.
5155/6670 completed.
5156/6670 completed.
5157/6670 completed.
5158/6670 completed.
5159/6670 completed.
5160/6670 completed.
5161/6670 completed.
5162/6670 completed.
5163/6670 completed.
5164/6670 completed.
5165/6670 completed.
5166/6670 completed.
5167/6670 completed.
5168/6670 completed.
5169/6670 completed.
5170/6670 completed.
5171/6670 completed.
5172/6670 completed.
5173/6670 completed.
5174/6670 completed.
5175/6670 completed.
5176/6670 completed.
5177/6670 completed.
5178/6670 completed.
5179/6670 completed.
5180/6670 completed.
5181/6670 completed.
5182/6670 completed.
5183/6670 completed.
5184/6670 completed.
5185/6670 completed.
5186/6670 completed.
5187/6670 completed.
5188/6670 completed.
5189/6670 completed.
5190/6670 completed.
5191/6670 completed.
5192/6670 completed.
5193/6670 completed.
5194/6670 completed.
5195/6670 completed.
5196/6670 completed.
5197/6670 completed.
5198/6670 completed.
5199/6670 completed.
5200/6670 completed.
5201/6670 com

5545/6670 completed.
5546/6670 completed.
5547/6670 completed.
5548/6670 completed.
5549/6670 completed.
5550/6670 completed.
5551/6670 completed.
5552/6670 completed.
5553/6670 completed.
5554/6670 completed.
5555/6670 completed.
5556/6670 completed.
5557/6670 completed.
5558/6670 completed.
5559/6670 completed.
5560/6670 completed.
5561/6670 completed.
5562/6670 completed.
5563/6670 completed.
5564/6670 completed.
5565/6670 completed.
5566/6670 completed.
5567/6670 completed.
5568/6670 completed.
5569/6670 completed.
5570/6670 completed.
5571/6670 completed.
5572/6670 completed.
5573/6670 completed.
5574/6670 completed.
5575/6670 completed.
5576/6670 completed.
5577/6670 completed.
5578/6670 completed.
5579/6670 completed.
5580/6670 completed.
5581/6670 completed.
5582/6670 completed.
5583/6670 completed.
5584/6670 completed.
5585/6670 completed.
5586/6670 completed.
5587/6670 completed.
5588/6670 completed.
5589/6670 completed.
5590/6670 completed.
5591/6670 completed.
5592/6670 com

5937/6670 completed.
5938/6670 completed.
5939/6670 completed.
5940/6670 completed.
5941/6670 completed.
5942/6670 completed.
5943/6670 completed.
5944/6670 completed.
5945/6670 completed.
5946/6670 completed.
5947/6670 completed.
5948/6670 completed.
5949/6670 completed.
5950/6670 completed.
5951/6670 completed.
5952/6670 completed.
5953/6670 completed.
5954/6670 completed.
5955/6670 completed.
5956/6670 completed.
5957/6670 completed.
5958/6670 completed.
5959/6670 completed.
5960/6670 completed.
5961/6670 completed.
5962/6670 completed.
5963/6670 completed.
5964/6670 completed.
5965/6670 completed.
5966/6670 completed.
5967/6670 completed.
5968/6670 completed.
5969/6670 completed.
5970/6670 completed.
5971/6670 completed.
5972/6670 completed.
5973/6670 completed.
5974/6670 completed.
5975/6670 completed.
5976/6670 completed.
5977/6670 completed.
5978/6670 completed.
5979/6670 completed.
5980/6670 completed.
5981/6670 completed.
5982/6670 completed.
5983/6670 completed.
5984/6670 com

6328/6670 completed.
6329/6670 completed.
6330/6670 completed.
6331/6670 completed.
6332/6670 completed.
6333/6670 completed.
6334/6670 completed.
6335/6670 completed.
6336/6670 completed.
6337/6670 completed.
6338/6670 completed.
6339/6670 completed.
6340/6670 completed.
6341/6670 completed.
6342/6670 completed.
6343/6670 completed.
6344/6670 completed.
6345/6670 completed.
6346/6670 completed.
6347/6670 completed.
6348/6670 completed.
6349/6670 completed.
6350/6670 completed.
6351/6670 completed.
6352/6670 completed.
6353/6670 completed.
6354/6670 completed.
6355/6670 completed.
6356/6670 completed.
6357/6670 completed.
6358/6670 completed.
6359/6670 completed.
6360/6670 completed.
6361/6670 completed.
6362/6670 completed.
6363/6670 completed.
6364/6670 completed.
6365/6670 completed.
6366/6670 completed.
6367/6670 completed.
6368/6670 completed.
6369/6670 completed.
6370/6670 completed.
6371/6670 completed.
6372/6670 completed.
6373/6670 completed.
6374/6670 completed.
6375/6670 com

In [22]:
generate_pair_df(data['AAN'], data['ATEN'], 30)

,date,year,monthOfYear,dayOfMonth,hourOfDay,minuteOfHour,dayOfWeek,dayOfYear,weekOfYear,isHoliday,...,spread,pvalue,open1,high1,low1,close1,open2,high2,low2,close2
30,2015-01-28,2015,1,28,0,0,2,28,5,0,...,0.018490,0.490233,32.650,32.855,32.195,32.290,5.255,5.345,4.825,4.860
31,2015-01-29,2015,1,29,0,0,3,29,5,0,...,0.055213,0.490233,32.450,32.640,32.200,32.395,4.820,4.945,4.590,4.700
32,2015-01-30,2015,1,30,0,0,4,30,5,0,...,0.010010,0.490233,32.215,32.400,31.515,31.655,4.535,4.875,4.530,4.805
33,2015-02-02,2015,2,2,0,0,0,33,6,0,...,0.021473,0.490233,31.745,32.030,30.545,31.920,4.815,4.835,4.725,4.790
34,2015-02-03,2015,2,3,0,0,1,34,6,0,...,-0.042927,0.490233,31.240,31.355,30.120,31.335,4.790,5.155,4.790,5.015
35,2015-02-04,2015,2,4,0,0,2,35,6,0,...,0.002864,0.490233,31.340,31.825,31.255,31.495,5.010,5.010,4.725,4.815
36,2015-02-05,2015,2,5,0,0,3,36,6,0,...,0.005013,0.490233,31.485,31.630,31.135,31.235,4.840,4.985,4.760,4.765
37,2015-02-06,2015,2,6,0,0,4,37,6,0,...,-0.001174,0.490233,30.585,33.660,30.585,32.215,4.780,5.015,4.780,4.945
38,2015-02-09,2015,2,9,0,0,0,40,7,0,...,-0.014477,0.490233,32.490,32.555,31.675,31.725,4.945,5.035,4.815,4.935
39,2015-02-10,2015,2,10,0,0,1,41,7,0,...,-0.028170,0.490233,31.860,31.860,30.895,31.135,4.955,4.975,4.690,4.910
